<img src="https://s3.cloud.cmctelecom.vn/tinhte1/2018/03/4267082_CV.jpg" width=browser_width >



# **Tiki Web Scraping with Selenium**


**Overview**: Build a web-crawler that take in a Tiki URL and return a dataframe 

**Due Date**: Before Monday next week.

**Requirements** 
1. Your function should be able to take in an URL and return a pandas dataframe
2. The final dataframe should contain the following informations: 
    * Product Name
    * Price
    * URL of the product image
    * URL of that product page

Try to follow the guideline below

#Install resources

In [ ]:
# install selenium and other resources for crawling data
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

     |████████████████████████████████| 904 kB 2.8 MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubu

In [ ]:
### IMPORTS ###
import re
import time
import pandas as pd

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

#Configuration for Driver and links

In [ ]:
###############
### GLOBALS ###
###############

# Urls
TIKI                = 'https://tiki.vn'
MAIN_CATEGORIES = [
    {'Name': 'Điện Thoại - Máy Tính Bảng',
     'URL': 'https://tiki.vn/dien-thoai-may-tinh-bang/c1789?src=c.1789.hamburger_menu_fly_out_banner'},

    {'Name': 'Điện Tử - Điện Lạnh',
     'URL': 'https://tiki.vn/tivi-thiet-bi-nghe-nhin/c4221?src=c.4221.hamburger_menu_fly_out_banner'},

    {'Name': 'Phụ Kiện - Thiết Bị Số', 
     'URL': 'https://tiki.vn/thiet-bi-kts-phu-kien-so/c1815?src=c.1815.hamburger_menu_fly_out_banner'},

    {'Name': 'Laptop - Thiết bị IT', 
     'URL': 'https://tiki.vn/laptop-may-vi-tinh/c1846?src=c.1846.hamburger_menu_fly_out_banner'},

    {'Name': 'Máy Ảnh - Quay Phim', 
     'URL': 'https://tiki.vn/may-anh/c1801?src=c.1801.hamburger_menu_fly_out_banner'},

    {'Name': 'Điện Gia Dụng', 
     'URL': 'https://tiki.vn/dien-gia-dung/c1882?src=c.1882.hamburger_menu_fly_out_banner'},

    {'Name': 'Nhà Cửa Đời Sống', 
     'URL': 'https://tiki.vn/nha-cua-doi-song/c1883?src=c.1883.hamburger_menu_fly_out_banner'},

    {'Name': 'Hàng Tiêu Dùng - Thực Phẩm', 
     'URL': 'https://tiki.vn/bach-hoa-online/c4384?src=c.4384.hamburger_menu_fly_out_banner'},

    {'Name': 'Đồ chơi, Mẹ & Bé', 
     'URL': 'https://tiki.vn/me-va-be/c2549?src=c.2549.hamburger_menu_fly_out_banner'},

    {'Name': 'Làm Đẹp - Sức Khỏe', 
     'URL': 'https://tiki.vn/lam-dep-suc-khoe/c1520?src=c.1520.hamburger_menu_fly_out_banner'},

    {'Name': 'Thể Thao - Dã Ngoại', 
     'URL': 'https://tiki.vn/the-thao/c1975?src=c.1975.hamburger_menu_fly_out_banner'},

    {'Name': 'Xe Máy, Ô tô, Xe Đạp', 
     'URL': 'https://tiki.vn/o-to-xe-may-xe-dap/c8594?src=c.8594.hamburger_menu_fly_out_banner'},

    {'Name': 'Hàng quốc tế', 
     'URL': 'https://tiki.vn/hang-quoc-te/c17166?src=c.17166.hamburger_menu_fly_out_banner'},

    {'Name': 'Sách, VPP & Quà Tặng', 
     'URL': 'https://tiki.vn/nha-sach-tiki/c8322?src=c.8322.hamburger_menu_fly_out_banner'},

    {'Name': 'Voucher - Dịch Vụ - Thẻ Cào', 
     'URL': 'https://tiki.vn/voucher-dich-vu/c11312?src=c.11312.hamburger_menu_fly_out_banner'}
]

# Global driver to use throughout the script
DRIVER = None

#Function to Start and Close Driver

In [ ]:
# Function to (re)start driver
def start_driver(force_restart=False):
    global DRIVER
    
    if DRIVER is not None:
        if force_restart:
            DRIVER.close()
        else:
            raise RuntimeError('ERROR: cannot overwrite an active driver. Please close the driver before restarting.')
    
    # Setting up the driver
    options = webdriver.ChromeOptions()
    options.add_argument('-headless') # we don't want a chrome browser opens, so it will run in the background
    options.add_argument('-no-sandbox')
    options.add_argument('-disable-dev-shm-usage')

    DRIVER = webdriver.Chrome('chromedriver',options=options)

# Wrapper to close driver if its created
def close_driver():
    global DRIVER
    if DRIVER is not None:
        DRIVER.close()
    DRIVER = None

#Function to get info from one product

In [ ]:
#################
### FUNCTIONS ###
#################


# Function to extract product info from the product
def get_product_info_single(product_item):
    d = {'name':'',
         'price':'',
         'product_url':'',
         'image':'',
         'FreeShip':'',
         'Rating':'',
         'Badge_Under_Price':'',
         'Be_Paid_By_Installments':'',
         'Free_Gifts':''
         }
    #.get_attribute('innerHTM') 
    regex_rating = r'(?<=\w\:\s)([\d]+\%)(?=\;)'
    # name get name through find_element_by_class_name
    try:
        name_tag = product_item.find_element_by_class_name('name')
        span_tag = name_tag.find_element_by_tag_name('span')
        d['name'] = span_tag.get_attribute('innerHTML') 
    except NoSuchElementException:
        pass

    # get price find_element_by_class_name
    try:
        d['price'] = product_item.find_element_by_class_name('price-discount__price').text
    except NoSuchElementException:
        d['price'] = -1
    if d['price'] == '':
        price_tag = product_item.find_element_by_class_name('price-discount__price')
        d['price'] = price_tag.get_attribute('innerHTML')
    
    # get link from .get_attribute()
    try:
        product_link = product_item.get_attribute('href')
        d['product_url'] = product_link.strip()
    except NoSuchElementException:
        pass
    # get thumbnail by class_name and Tag name and get_attribute()
    try:
        thumbnail =  product_item.find_element_by_class_name('thumbnail')
        d['image'] = thumbnail.find_element_by_tag_name('source').get_attribute('srcset')
    except NoSuchElementException:
        pass
    # get FreeShip by class_name and Tag name and get_attribute()
    try:
        d['FreeShip'] = thumbnail.find_element_by_tag_name('img').get_attribute('src')
    except NoSuchElementException:
        pass
    # get Rating by class_name 'full-rating' and class name 'average' and get_attribute()
    try:
        Rating = product_item.find_element_by_class_name('full-rating')
        Rating_average = Rating.find_element_by_class_name('average')
        Full_Rating = Rating_average.get_attribute('style')
        d['Rating'] = re.search(regex_rating,Full_Rating).group()
    except NoSuchElementException:
        pass
    # get Badge_Under_Price by class_name 'badge-under-price' and by tag name 'img' then get_attribute()
    try:
        Badge_Under_Price = product_item.find_element_by_class_name('badge-under-price')
        Badge_Under_Price_src = Badge_Under_Price.find_element_by_tag_name('img')
        Badge_Under_Price_Check = Badge_Under_Price_src.get_attribute('src')
        if Badge_Under_Price_Check != '':
            d['Badge_Under_Price'] = True
        else:
            d['Badge_Under_Price'] = False
    except NoSuchElementException:
        pass
    # get Be_Paid_By_Installments by class_name 'badge-benefits' and by tag name 'img' then get_attribute()
    try:
        Be_Paid_By_Installments = product_item.find_element_by_class_name('badge-benefits')
        Be_Paid_By_Installments_src = Be_Paid_By_Installments.find_element_by_tag_name('img')
        Be_Paid_By_Installments_Check = Be_Paid_By_Installments_src.get_attribute('src')
        if Be_Paid_By_Installments != '':
            d['Be_Paid_By_Installments'] = True
        else:
            d['Be_Paid_By_Installments'] = False
    except NoSuchElementException:
        pass
    # get Free_Gifts by class_name 'freegift-list' and by tag name 'img' then get_attribute()
    try:
        Free_Gifts = product_item.find_element_by_class_name('freegift-list')
        Free_Gifts_src = Free_Gifts.find_element_by_tag_name('img')
        Free_Gifts_Check = Free_Gifts_src.get_attribute('src')
        if Free_Gifts != '':
            d['Free_Gifts'] = True
        else:
            d['Free_Gifts'] = False
    except NoSuchElementException:
        pass
    return d


#Function to scrape info of all products from a Page URL

In [ ]:
###########################################################
#######FUNCTIONS TO TAKE ALL THE SMALL PAGES LIMITED ######
###########################################################
def get_all_smalls_page_limit(page_url,max_page):
    global DRIVER
    DRIVER.get(page_url)
    time.sleep(5)
    #Regex to take the page number
    regex_page_number = r'(?<=\?page\=)(\d+)(?=&src=c\.\d+\.hamburger\_menu\_fly_out\_banner)'
    #Try to take the element which inlucde all the tags have product_list_pagination_item
    try:
        tag_include_product_pages = DRIVER.find_element_by_tag_name('ul')
        product_list_page = tag_include_product_pages.find_elements_by_tag_name('a')
    except NoSuchElementException:
        return List_all_page_number
    #Find page number in page_url bu regex
    current_page_number = re.search(regex_page_number,page_url)
    #URL already include page number
    if current_page_number == None:
        current_page_number = 1
        current_page_find = tag_include_product_pages.find_element_by_class_name('current')
        current_page = current_page_find.get_attribute('href')
        # Creat an List to store all the link of the small pages   
        List_all_page_number = [page_url]
    # Create for loops to store all the page url in product_list_pagination_item tag
    for page in range(2,max_page+1):
        #next_pages = re.sub(regex_page_number, str(page),current_page)
        List_all_page_number.append(re.sub(regex_page_number, str(page),current_page))
    return List_all_page_number

In [ ]:
# Function to scrape all products from a page
def get_product_info_from_page(page_url):
    """ Extract info from all products of a specfic page_url on Tiki website
        Args:
            page_url: (string) url of the page to scrape
        Returns:
            data: list of dictionary of products info. If no products shown, return empty list.
    """
    global DRIVER

    data = []
    DRIVER.get(page_url) # Use the driver to get info from the product page
    time.sleep(5)        # MUST have the sleep function
    count_products = 0
    products_all = DRIVER.find_elements_by_class_name('product-item')
    #print(f'Found {len(products)} products')

    for product in products_all:
      # Look through the product and get the data
      # YOUR CODE HERE
      data_check = get_product_info_single(product)
      if data_check['name'] != '' and data_check['price'] != '':
            data.append(data_check)
            count_products += 1
    print(f'Found {count_products} products')
    return data

In [ ]:
######################
### START SCRAPING ###
######################
num_max_page = 5
# main_cat_url = MAIN_CATEGORIES[-1]['URL']

start_driver(force_restart=True)
prod_data = []
List_Small_Page = []
#CODE TO GET DATA
for i in range (0,len(MAIN_CATEGORIES)):
    current_name = MAIN_CATEGORIES[i]['Name']
    List_Small_Page = get_all_smalls_page_limit(MAIN_CATEGORIES[i]['URL'],num_max_page)
    print(f'Take {len(List_Small_Page)} small page in {current_name}')
    for k in List_Small_Page:
        prod_data.extend(get_product_info_from_page(k))
close_driver()

#SAVE DATA TO CSV FILE
df = pd.DataFrame(data=prod_data, columns=prod_data[0].keys())
df.to_csv('tiki_products.csv')


In [ ]:
df.head()

,name,price,product_url,image,FreeShip,Rating,Badge_Under_Price,Be_Paid_By_Installments,Free_Gifts
0,Máy Tính Bảng Huawei MatePad 11 | Màn Hình HUA...,16.480.000 ₫,https://tka.tiki.vn/pixel?CLICK=&data=djAwMSYv...,https://salt.tikicdn.com/cache/200x200/ts/prod...,https://salt.tikicdn.com/ts/upload/dc/0d/49/32...,,,True,
1,Điện Thoại Samsung Galaxy M12 (4GB/64GB) - Hàn...,3.639.000 ₫,https://tiki.vn/dien-thoai-samsung-galaxy-m12-...,https://salt.tikicdn.com/cache/200x200/ts/prod...,https://salt.tikicdn.com/ts/upload/dc/0d/49/32...,96%,True,True,True
2,Điện Thoại Vsmart Joy 4 - Hàng Chính Hãng,3.059.000 ₫,https://tiki.vn/dien-thoai-vsmart-joy-4-hang-c...,https://salt.tikicdn.com/cache/200x200/ts/prod...,https://salt.tikicdn.com/ts/upload/dc/0d/49/32...,94%,,,
3,Điện thoại Vivo Y12s - Hàng Chính Hãng,3.789.000 ₫,https://tiki.vn/dien-thoai-vivo-y12s-hang-chin...,https://salt.tikicdn.com/cache/200x200/ts/prod...,https://salt.tikicdn.com/ts/upload/dc/0d/49/32...,96%,True,,
4,Điện Thoại Samsung Galaxy Z Fold 3 (512GB) - H...,44.990.000 ₫,https://tka.tiki.vn/pixel?CLICK=&data=djAwMWxi...,https://salt.tikicdn.com/cache/200x200/ts/prod...,https://salt.tikicdn.com/ts/upload/dc/0d/49/32...,80%,,True,True


**Extra Optional Requirement**


Bonus information:

* Does it has FreeShip? <img src="https://salt.tikicdn.com/ts/upload/dc/0d/49/3251737db2de83b74eba8a9ad6d03338.png">
* Number of reviews?
* Number of sale?
* How many stars or percentage of stars?
* Does it got "badge under price" (Rẻ hơn hoàn tiền)? <img src="https://salt.tikicdn.com/ts/upload/51/ac/cc/528e80fe3f464f910174e2fdf8887b6f.png">
* Discount percentage?
* Does it got "shocking price" badge? <img src="https://salt.tikicdn.com/ts/upload/75/34/d2/4a9a0958a782da8930cdad8f08afff37.png">
* Does it allowed to be paid by installments? <img src="https://salt.tikicdn.com/ts/upload/ba/4e/6e/26e9f2487e9f49b7dcf4043960e687dd.png">
* Does it comes with free gifts? <img src="https://salt.tikicdn.com/ts/upload/47/35/8c/446f61d046eba9a305d3f39dc0834c4a.png">
    


In [ ]:
###################################################
#######FUNCTIONS TO TAKE ALL THE SMALL PAGES ######
###################################################
def get_all_smalls_page(page_url):
    global DRIVER
    global List_page_number
    global next_page
    DRIVER.get(page_url)
    time.sleep(5)
    #Regex to take the page number
    regex_page_number = r'(?<=\?page\=)(\d+)(?=&src=c\.\d+\.hamburger\_menu\_fly_out\_banner)'
    current_page = page_url
    #Try to take the element which inlucde all the tags have product_list_pagination_item
    try:
        tag_include_product_pages = DRIVER.find_element_by_tag_name('ul')
        product_list_page = tag_include_product_pages.find_elements_by_tag_name('a')
    except NoSuchElementException:
        return List_page_number
    #Find page number in page_url bu regex
    current_page_number = re.search(regex_page_number,page_url)
    #URL already include page number
    if current_page_number != None:
        current_page_number = int(re.search(regex_page_number,page_url).group())
    #Replace URL with URL included page number for the first page
    else:
        current_page_number = 1
        current_page_find = tag_include_product_pages.find_element_by_class_name('current')
        current_page = current_page_find.get_attribute('href')
        # Creat an List to store all the link of the small pages   
        List_page_number = [page_url]
    # if current_page_number > max_page:
    #     return List_page_number
    # List to store all the small pages of product_list_pagination_item tag
    # List_page =[]
    # Create for loops to store all the page number in product_list_pagination_item tag
    for page in product_list_page:
        number_page = int(page.get_attribute('data-view-label'))
        # if(number_page not in List_page):
        #     List_page.append(number_page)
        #Find the next page which appear twice in product_list_pagination_item tag and page number greater than current page
        if(number_page == current_page_number + 1):
            # Store information of next page
            next_page = re.sub(regex_page_number, str(number_page),current_page)
            # Append url to List_page_number
            List_page_number.append(next_page)
            # Recursive function to go to the next page and take url
            # Stop when there is no next page
            get_all_smalls_page(next_page)
            time.sleep(1)
            break
    return List_page_number

In [ ]:
#############################################################################
### START SCRAPING ALLL THE PAGES OF 'Name': 'Điện Thoại - Máy Tính Bảng' ###
#############################################################################
start_driver(force_restart=True)
# Creat a list to take all product of TIKI
List_all_TIKI_products = []
# Creat a list to store all small pages of page 'Điện Thoại - Máy Tính Bảng'
List_all_small_pages_first_element = []
# First URL of MAIN_CATEGORIES[0]['URL']
List_all_small_pages_first_element = get_all_smalls_page(MAIN_CATEGORIES[0]['URL'])

print(f'There are {len(List_all_small_pages_first_element)} small pages in Điện Thoại - Máy Tính Bảng ')
# Traverse all small pages in List_all_small_pages_first_element
for small_page_1 in List_all_small_pages_first_element:
    List_all_TIKI_products.extend(get_product_info_from_page(small_page_1))
print(f'There are {len(List_all_TIKI_products)} products in Điện Thoại - Máy Tính Bảng ')
print(List_all_small_pages_first_element[-1])

There are 8 small pages in Điện Thoại - Máy Tính Bảng 
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 71 products
Found 71 products
Found 72 products
Found 44 products
There are 546 products in Điện Thoại - Máy Tính Bảng 
https://tiki.vn/dien-thoai-may-tinh-bang/c1789?page=8&src=c.1789.hamburger_menu_fly_out_banner


In [ ]:
close_driver()
print(List_all_small_pages_first_element[-1])
#SAVE DATA TO CSV FILE
df = pd.DataFrame(data=List_all_TIKI_products, columns=List_all_TIKI_products[0].keys())
df.to_csv('tiki_products_Dien_Thoai_May_Tinh_Bang.csv')
df.head()

,name,price,product_url,image,FreeShip,Rating,Badge_Under_Price,Be_Paid_By_Installments,Free_Gifts
0,Điện Thoại Samsung Galaxy M12 (4GB/64GB) - Hàn...,3.609.000 ₫,https://tka.tiki.vn/pixel?CLICK=&data=djAwMfp4...,https://salt.tikicdn.com/cache/200x200/ts/prod...,https://salt.tikicdn.com/ts/upload/dc/0d/49/32...,96%,True,True,True
1,Điện Thoại Samsung Galaxy M12 (4GB/64GB) - Hàn...,3.609.000 ₫,https://tiki.vn/dien-thoai-samsung-galaxy-m12-...,https://salt.tikicdn.com/cache/200x200/ts/prod...,https://salt.tikicdn.com/ts/upload/dc/0d/49/32...,96%,True,True,True
2,Điện Thoại Vsmart Joy 4 - Hàng Chính Hãng,3.059.000 ₫,https://tiki.vn/dien-thoai-vsmart-joy-4-hang-c...,https://salt.tikicdn.com/cache/200x200/ts/prod...,https://salt.tikicdn.com/ts/upload/dc/0d/49/32...,94%,,,
3,Điện thoại Vivo Y12s - Hàng Chính Hãng,3.789.000 ₫,https://tiki.vn/dien-thoai-vivo-y12s-hang-chin...,https://salt.tikicdn.com/cache/200x200/ts/prod...,https://salt.tikicdn.com/ts/upload/dc/0d/49/32...,96%,True,,
4,Điện Thoại Samsung Galaxy Note 10 (8GB/256GB) ...,12.590.000 ₫,https://tka.tiki.vn/pixel?CLICK=&data=djAwMXWL...,https://salt.tikicdn.com/cache/200x200/ts/prod...,https://salt.tikicdn.com/ts/upload/dc/0d/49/32...,92%,,True,


In [ ]:
######################################################################
### START SCRAPING ALLL THE PAGES OF 'Name': 'Điện Tử - Điện Lạnh' ###
######################################################################
start_driver(force_restart=True)
# Creat a list to take all product of TIKI
List_all_TIKI_products_2 = []
# Creat a list to store all small pages of page 'Điện Tử - Điện Lạnh'
List_all_small_pages_second_element = []
# First URL of MAIN_CATEGORIES[1]['URL']
List_all_small_pages_second_element = get_all_smalls_page(MAIN_CATEGORIES[1]['URL'])

print(f'There are {len(List_all_small_pages_second_element)} small pages in Điện Tử - Điện Lạnh ')
# Traverse all small pages in List_all_small_pages_second_element
for small_page_2 in List_all_small_pages_second_element:
    List_all_TIKI_products_2.extend(get_product_info_from_page(small_page_2))
print(f'There are {len(List_all_TIKI_products_2)} products in Điện Tử - Điện Lạnh ')
print(List_all_small_pages_second_element[-1])
close_driver()

#SAVE DATA TO CSV FILE
df = pd.DataFrame(data=List_all_TIKI_products_2, columns=List_all_TIKI_products_2[0].keys())
df.to_csv('tiki_products_Dien_Tu_Dien_Lanh.csv')
df.head()

There are 209 small pages in Điện Tử - Điện Lạnh 
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 71 products
Found 71 products
Found 72 products
Found 72 products
Found 71 products
Found 71 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 products
Found 72 produ

,name,price,product_url,image,FreeShip,Rating,Badge_Under_Price,Be_Paid_By_Installments,Free_Gifts
0,Máy giặt Samsung Inverter 8 kg WW80T3020WW - C...,5.159.000 ₫,https://tka.tiki.vn/pixel?CLICK=&data=djAwMRhI...,https://salt.tikicdn.com/cache/200x200/ts/prod...,https://salt.tikicdn.com/ts/upload/dc/0d/49/32...,92%,,True,
1,Tủ Lạnh Aqua 130 lít AQR-T150FA-BS,3.719.000 ₫,https://tiki.vn/tu-tu-lanh-aqua-130-lit-aqr-t1...,https://salt.tikicdn.com/cache/200x200/ts/prod...,https://salt.tikicdn.com/cache/200x200/ts/prod...,,,,
2,Tủ lạnh Samsung Inverter 208 lít RT19M300BGS/S...,4.749.000 ₫,https://tiki.vn/tu-lanh-samsung-inverter-208-l...,https://salt.tikicdn.com/cache/200x200/ts/prod...,https://salt.tikicdn.com/ts/upload/dc/0d/49/32...,92%,True,True,
3,Smart Tivi Casper HD 32 inch 32HX6200,4.199.900 ₫,https://tiki.vn/smart-tivi-casper-hd-32-inch-3...,https://salt.tikicdn.com/cache/200x200/ts/prod...,https://salt.tikicdn.com/ts/upload/dc/0d/49/32...,92%,True,True,
4,Tủ Lạnh Inverter Samsung RT20HAR8DBU/SV (208L)...,4.909.000 ₫,https://tka.tiki.vn/pixel?CLICK=&data=djAwMU5v...,https://salt.tikicdn.com/cache/200x200/ts/prod...,https://salt.tikicdn.com/ts/upload/dc/0d/49/32...,92%,,True,
